## X-ray tomography and diffraction

We begin by loading the libraries you will require later on for your calculations.

In [4]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

# Add the parent directory including test_simulation.py to the system path
sys.path.append(os.path.abspath("../data/"))

from test_simulation_adapted import forward_propagation_holo

---
### Exercise 1

In this exercise, you will use a function to simulate the forward propagation of X-rays after interacting with an object, employing the Fresnel propagator.

**Important Notice:**
- The function in `test_simulation.py` does not fall under the MIT license and may not be shared outside of this course.

**Tasks:**
1. Previously, you varied the inputs $ z_{01} $ and $ z_{02} $ (in nm) that relate to the Source-to-Object Distance (SOD) and Source-to-Detector Distance (SDD) in cone-beam imaging, and observed how the propagated image changes.
2. This week, modify the simulation to use materials such as **gold** and **Teflon**. 
3. Vary the **thickness** of these materials and observe how the propagated image is affected.
4. You may also experiment by using **two shapes of different materials** as the simulated object. Modify the function accordingly.

**Additional Resources:**
To obtain values for **beta** and **delta** based on beam energy and the materials chosen, refer to the following links:
- [Optical Constants - Perturbed Form](https://henke.lbl.gov/optical_constants/pert_form.html)
- [Optical Constants - Database Access](https://henke.lbl.gov/optical_constants/getdb2.html)


---
#### Solution
_- solve the exercise beneath using markdown and/or code blocks -_

In [ ]:
# Optical constants (delta and beta) for materials at 11 keV
optical_constants = {
    "gold": (2.42117549e-05, 1.56585247e-06),  # (delta, beta) for gold
    "teflon": (3.638105e-06, 9.52290691e-09),  # (delta, beta) for teflon
}

# Propagation distances (converted to nm) and material thicknesses (in meters)
distances_nm = {
    "z01": 10 * 1e7,  # 10 cm = 10 * 1e7 nm
    "z02_far": 10 * 1e9,  # 20 m = 10 * 1e9 nm
    "z02_near": 20 * 1e7,  # 20 cm = 20 * 1e7 nm
}
thicknesses = {
    "thin": 20e-6,  # 20 µm
    "thick": 200e-6,  # 200 µm
}

# Run simulations for gold
au_results = {}
delta, beta = optical_constants["gold"]  # Get optical constants for specified material
au_results["far_thick"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_far"], thicknesses["thick"], delta, beta
)
au_results["far_thin"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_far"], thicknesses["thin"], delta, beta
)
au_results["near_thin"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_near"], thicknesses["thin"], delta, beta
)

# Run simulations for teflon
tf_results = {}
delta, beta = optical_constants[
    "teflon"
]  # Get optical constants for specified material
tf_results["far_thick"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_far"], thicknesses["thick"], delta, beta
)
tf_results["far_thin"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_far"], thicknesses["thin"], delta, beta
)
tf_results["near_thin"] = forward_propagation_holo(
    distances_nm["z01"], distances_nm["z02_near"], thicknesses["thin"], delta, beta
)

# Extract results for further processing if needed
au_far_thick, obj_au_thick = au_results["far_thick"]
au_far_thin, obj_au_thin = au_results["far_thin"]
au_near_thin, obj_au_near = au_results["near_thin"]

tf_far_thick, obj_tf_thick = tf_results["far_thick"]
tf_far_thin, obj_tf_thin = tf_results["far_thin"]
tf_near_thin, obj_tf_near = tf_results["near_thin"]


In [ ]:
# Create a grid of subplots to display results
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(8, 5))

# Titles and corresponding image data
titles = [
    "Au thick far",
    "Au thin far",
    "Au thin near",
    "Teflon thick far",
    "Teflon thin far",
    "Teflon thin near",
]

image_data = [
    au_far_thick,
    au_far_thin,
    au_near_thin,
    tf_far_thick,
    tf_far_thin,
    tf_near_thin,
]

# Plot images on each subplot
for ax, title, data in zip(axes.flatten(), titles, image_data):
    ax.imshow(data, vmin=0.1, vmax=1.15, cmap="Greys_r")
    ax.set_title(title)

# Adjust layout for better spacing
fig.tight_layout()
plt.show()


In the images of thin materials with low Sample-to-Detector Distance (SDD) (**thin near**), we observe absorption characteristics: Teflon shows low absorption, while gold exhibits high absorption.

As we increase the SDD (**thin far**), the phase shift and resulting interference produce a strong signal at the detector for both materials.

Interestingly, when we increase the sample thickness (**thick far**), we note no change in the signal for gold, which indicates an error as the transmission approaches zero. Conversely, the signal for Teflon appears weaker, likely due to phase wrapping effects.

---
### Exercise 2

A binary Fresnel zone plate has an outer zone width of 50 nm and a depth of 20 times the width, filled with gold (Au). The lens delivers an absolute phase shift of $ k\delta l = \pi $, where $ k $ is the wave number, $ \delta $ is the refractive index increment, and $ l $ is the depth of the zone plate. The diameter $ D $ of the lens is 100 µm. 

**Tasks:**
1. Determine the wavelength $ \lambda $.
2. Calculate the first-order focal length $ f $.
3. Estimate the depth of focus.

**Assumptions and Given Data:**
- The real part of the refractive index increment for purely scattering media is given by:
  $
  \delta = \frac{2\pi \rho r_0}{k^2}
  $
  where:
  - $ \rho = 4.687 $ electrons/Å$^3$ (electron density of Au),
  - $ r_0 = 2.82 \times 10^{-5} $ Å (scattering amplitude per electron).
- The depth of focus can be estimated as the resolution divided by the aperture of the zone plate, where the aperture $ A = D/f $.

**Reference:**
This question is adapted from *Elements of Modern X‐ray Physics* by J. Als-Nielsen and D. McMorrow (Second Edition, Chapter 9, DOI: 10.1002/9781119998365).


---
#### Solution
_- solve the exercise beneath using markdown and/or code blocks -_

1. **Calculating the Number of Zones:**  
   The number of zones $ M $ can be computed using the formula:
   $$
   D = 4M \cdot \Delta r_M
   $$
   Rearranging gives:
   $$
   M = \frac{D}{4 \cdot \Delta r_M} = 500
   $$

2. **Relating Wavelength, Focal Length, and Number of Zones:**  
   The wavelength $ \lambda $ and the first-order focal length $ f $ relate to the number of zones $ M $ and the outer zone width $ \Delta r_M $ as follows:
   $$
   \lambda \cdot f = 4M \cdot \Delta r_M^2
   $$

3. **Determining the Wavelength $ \lambda $:**  
   The Fresnel zone plate introduces a phase shift of $ \left|\Delta\Phi\right| = \pi $. This phase shift can be expressed as:
   $$
   \left|\Delta\Phi\right| = k \cdot \delta \cdot l
   $$
   where:
   - $ k $ is the wavenumber,
   - $ \delta $ is the real part of the refractive index increment,
   - $ l $ is the depth of the Au-filled zone plate.

   Thus, we have:
   $$
   \left|\Delta\Phi\right| = \pi = k \cdot \delta \cdot l = \frac{2k \cdot l \cdot \pi \cdot \rho \cdot r_0}{k^2}
   $$
   Substituting $ k = \frac{2\pi}{\lambda} $ gives:
   $$
   \pi = \frac{2\lambda \cdot l \cdot \pi \cdot \rho \cdot r_0}{2\pi}
   $$
   Rearranging results in:
   $$
   \lambda = \frac{\pi}{l \cdot \rho \cdot r_0} = 2.38 \, \text{Å}
   $$

4. **Calculating the First-order Focal Length $ f $:**  
   Substitute $ \lambda $ back into the equation:
   $$
   f = \frac{4M \cdot \Delta r_M^2}{\lambda} = 21 \, \text{mm}
   $$

5. **Determining the Resolution and Depth of Focus:**  
   The resolution of the zone plate is given by:
   $$
   \Delta x = 1.22 \cdot \Delta r_M = 61 \, \text{nm}
   $$
   The aperture $ A $ is calculated as:
   $$
   A = \frac{D}{f} = 4.76 \times 10^{-3}
   $$
   Finally, the depth of focus is:
   $$
   \frac{\Delta x}{A} = 12.8 \, \mu\text{m}
   $$


In [ ]:
# Given constants
D = 100e-6  # Diameter in meters (100 µm)
delta_r_M = 50e-9  # Outer zone width in meters (50 nm)
rho = 4.687 / (
    1e-10**3
)  # Electron density of Au in electrons/m^3 (converted from electrons/Å^3)
r0 = 2.82e-5 * 1e-10  # Scattering amplitude per electron in meters (converted from Å)
l = 20 * delta_r_M  # Depth of the zone plate (20 times the outer zone width)

# 1. Calculate the number of zones M
M = D / (4 * delta_r_M)

# Calculate real part of refractive index delta
# delta = 2*np.pi*rho*r0/k**2
# k = np.pi/(delta*l)
# Calculate wave No. k
k = 2 * rho * r0 * l

# 2. Calculate the wavelength λ
lmbda = 2 * np.pi / k

# 3. Calculate focal length f of first order
f = 4 * M * delta_r_M**2 / lmbda

# 4. Calculate the resolution Δx
delta_x = 1.22 * delta_r_M

# 5. Calculate the aperture A
A = D / f

# 6. Calculate the depth of focus z_f
z_f = delta_x / A

# Display results
print(f"Number of Zones M: {M:.0f}")
print(f"Wavelength λ: {lmbda:.2e} m")
print(f"Wavelength λ: {lmbda*1e10:.2f} Å")
print(f"First-order Focal Length f: {f*1e3:.2f} mm")
print(f"Resolution Δx: {delta_x * 1e9:.2f} nm")
print(f"Aperture A: {A:.2e}")
print(f"Depth of Focus: {z_f*1e6:.2f} µm")


---
### Exercise 3

The file `AMS_DATA_zincite.txt` contains data on the diffraction properties of ZnO (zincite) from the AMS database. The d-spacing values are provided in Ångströms (Å).

**Tasks:**
1. Using a line detector, compute the required **lateral translation** for various detector distances, ranging from **0.5 cm to 10 cm** in increments of **0.5 cm**.
2. Ensure that the (200) reflection is measured in the middle of the detector for photon energies between **5 keV and 80 keV** in **1 keV steps**.
3. By default, the center of the detector is aligned with the undiffracted beam.
4. Create a **3D plot** showing the relationship between the detector distances and the photon energies.

**Given Constants:**
- Speed of light, $ c = 3 \times 10^8 \, \text{m/s} $
- Planck's constant, $ h = 4.136 \times 10^{-15} \, \text{eV} \cdot \text{s} $

**Hint:**
- You have to read the d-spacing from the `AMS_DATA_zincite.txt` file
- Inform about **Miller indices** (hkl) and use [Link](http://calistry.org/calculate/latticePlanesMillerIndices) to visualize the reflection adressed in the excercise

---
#### Solution
_- solve the exercise beneath using markdown and/or code blocks -_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# speed of light in m/s
c = 3 * 10 ** (8)
# Planck's constant in eV s
h = 4.136 * 10 ** (-15)

# d-spacing in m read from the .txt file
d_spacing = 1.3943 * 10 ** (-10)
# assign energy range in eV
eV = np.arange(5000, 81000, 1000)
# assign different propagation distances in m
z = np.arange(0.005, 0.1, 0.005)

# generate meshes for the distance and E,
# such that the xy translation
# can be calculated for all combinations of both

Z, E = np.meshgrid(z, eV)
# compute xy translation by using Bragg's law to
# determine the scattering angle at each energy.
# lambda=2*d*np.sin(theta)
# use geometric consideration to determine xy
# translation via the tangens of the propagation distance
# and the scattering angle. Note that a diffraction
# of an angle 2*theta larger than
# 90 degrees is not possible, we therefore must
# omit these instances.
theta = np.arcsin((c * h / E) / (2 * d_spacing))
XY = np.tan(2 * theta) * Z
XY[np.where(np.rad2deg(theta) > 45)] = np.nan

# generate surface plot
fig, ax = plt.subplots(subplot_kw=dict(projection="3d"))
surf = ax.plot_surface(Z * 100, E / 1000, XY * 100, alpha=0.8, cmap=cm.gist_earth)
plt.xlabel("Propagation distance / cm ")
plt.ylabel("Energy / keV")
ax.set_zlabel("Lateral translation / cm")
plt.title("Required lateral translation of detector to place ZnO 200 centrally")
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
ax.view_init(30, 45)
plt.show()